In [34]:
import os
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

In [35]:
import struct
 
def load_mnist(path, kind='train'):
    labels_path = os.path.join(path, '%s-labels.idx1-ubyte' % kind).replace('\\','/')
    images_path = os.path.join(path, '%s-images.idx3-ubyte' % kind).replace('\\','/')
        
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", 
                                               imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
        images = ((images / 255.) - .5) * 2
 
    return images, labels

In [36]:
current_path = os.path.join(os.getcwd(),'mnist')
X_train, y_train = load_mnist(current_path, kind='train')
print('Rows: %d,  Columns: %d' %(X_train.shape[0], X_train.shape[1]))

Rows: 60000,  Columns: 784


In [37]:
X_test, y_test = load_mnist(current_path, kind='t10k')
print('Rows: %d, columns: %d' % (X_test.shape[0], X_test.shape[1]))

Rows: 10000, columns: 784


In [38]:
# 平均&標準化
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

# 中心化
X_train_centered = (X_train - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val

del X_train, X_test

print(X_train_centered.shape, y_train.shape)
print(X_test_centered.shape, y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [39]:
np.random.seed(123)
tf.set_random_seed(19)

In [40]:
# kerasのワンホット表現
y_train_onehot = keras.utils.to_categorical(y_train)
 
print('First 3 labels: ', y_train[:3])
print('\nFirst 3 labels (one-hot):\n', y_train_onehot[:3])

First 3 labels:  [5 0 4]

First 3 labels (one-hot):
 [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


### Kerasによるニューラルネットワークの作成

In [41]:
model = keras.models.Sequential()

model.add(
    keras.layers.Dense(
        units=50,    
        input_dim=X_train_centered.shape[1],
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))

model.add(
    keras.layers.Dense(
        units=50,    
        input_dim=50,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))

model.add(
    keras.layers.Dense(
        units=y_train_onehot.shape[1],    
        input_dim=50,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='softmax'))


sgd_optimizer = keras.optimizers.SGD(lr=0.001, decay=1e-7, momentum=.9)

model.compile(optimizer=sgd_optimizer,loss='categorical_crossentropy')

In [42]:
history = model.fit(X_train_centered, y_train_onehot,
                    batch_size=64, epochs=50,
                    verbose=1,
                    validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 2s 38us/sample - loss: 0.7406 - val_loss: 0.3680
Epoch 2/50
54000/54000 [==============================] - 2s 38us/sample - loss: 0.3747 - val_loss: 0.2778
Epoch 3/50
54000/54000 [==============================] - 2s 34us/sample - loss: 0.3080 - val_loss: 0.2396
Epoch 4/50
54000/54000 [==============================] - 2s 36us/sample - loss: 0.2715 - val_loss: 0.2163
Epoch 5/50
54000/54000 [==============================] - 2s 37us/sample - loss: 0.2461 - val_loss: 0.2005
Epoch 6/50
54000/54000 [==============================] - 2s 35us/sample - loss: 0.2265 - val_loss: 0.1857
Epoch 7/50
54000/54000 [==============================] - 2s 38us/sample - loss: 0.2103 - val_loss: 0.1759
Epoch 8/50
54000/54000 [==============================] - 2s 34us/sample - loss: 0.1966 - val_loss: 0.1673
Epoch 9/50
54000/54000 [==============================] - 2s 34us/sample - loss: 0.1846 - val_l

In [43]:
y_train_pred = model.predict_classes(X_train_centered, verbose=0)
print('First 3 predictions: ', y_train_pred[:3])

First 3 predictions:  [5 0 4]


In [44]:
y_train_pred = model.predict_classes(X_train_centered, 
                                     verbose=0)
correct_preds = np.sum(y_train == y_train_pred, axis=0) 
train_acc = correct_preds / y_train.shape[0]

print('First 3 predictions: ', y_train_pred[:3])
print('Training accuracy: %.2f%%' % (train_acc * 100))

First 3 predictions:  [5 0 4]
Training accuracy: 99.00%


In [45]:
y_test_pred = model.predict_classes(X_test_centered, 
                                    verbose=0)

correct_preds = np.sum(y_test == y_test_pred, axis=0) 
test_acc = correct_preds / y_test.shape[0]
print('Test accuracy: %.2f%%' % (test_acc * 100))

Test accuracy: 96.55%
